# 0. Dependencies, load datasets

Определим все необходимые зависимости

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D 
import math

from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing 
from sklearn import neighbors, tree, ensemble, linear_model

import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = '{:.1f}'.format

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
subm_df_raw = pd.read_csv('../input/sample_submission.csv')

Поскольку в данных присутствует дата, сразу перобразуем ее в месяц, год и день недели, которые в дальнейшем можно использовать как фичи.

In [ ]:
train_df['date'] = pd.to_datetime(train_df.date)
train_df['month'] = train_df.date.dt.month
train_df['year'] = train_df.date.dt.year
train_df['dayofweek'] = train_df.date.dt.dayofweek

test_df['date'] = pd.to_datetime(test_df.date)
test_df['month'] = test_df.date.dt.month
test_df['year'] = test_df.date.dt.year
test_df['dayofweek'] = test_df.date.dt.dayofweek

test_df = test_df.drop(['id'], 1)

# 1. Data Analysis

### 1.1 First look

При первом взгляде на данные при помощи функций info(), describe() можно определить какие фичи присутствуют в датасете, какого они типа, какое количество строк, есть ли пропущенные значения и аномалии (сравнение максимального значения, мат. ожидания и процентилей из функции describe()).

In [ ]:
print('train_df info')
print(train_df.info())
print('')
print('train_df describe')
print(train_df.describe())
print('')
print('test_df info')
print(test_df.info())
print('')
print('test_df describe')
print(test_df.describe())

Как видно из функций, в массивах нет пропущеннх данных и аномалий. Все данные в численных значениях (кроме даты, из которой мы вытащили необходимые данные в числах).

### 1.2 Graphs

Для анализа имеющихся данных можно вывести графики изменения параметров от разных значений. Поскольку мы имеем дело с продажами 50 единиц товаров в 10 магазинах, можно посмотреть следующие зависимости:
* как изменяются продажи в зависимости от типа товара
* как изменяются продажи в зависимости от магазина
* как изменяются продажи в зависимости от года

In [ ]:
plt.figure(figsize=(7, 10))

plt.subplot(311)
plt.xticks(range(0, 55, 5))
plt.grid()
plt.title('Sales by item')
_ = plt.plot(train_df.groupby(['item'])['sales'].sum())

plt.subplot(312)
plt.xticks(range(1, 11))
plt.grid()
plt.title('Sales by store')
_ = plt.plot(train_df.groupby(['store'])['sales'].sum())

plt.subplot(313)
plt.xticks(range(2013, 2018))
plt.grid()
plt.title('Sales by year')
_ = plt.plot(train_df.groupby(train_df.date.dt.year)['sales'].sum())

Итого: 
* общие продажи в зависимости от типа товара имеют довольно случайный характер и нельзя выделить какой то полезной корреляции. В качестве обработки фич можно было бы выделить три корзины (binning) - товары которые плохо продаются, средние продажи и высокие продажи, но поскольку у нас нет проблемы большого разброса продажи отдельных позиций, нет необходимости в такой обработке. Тем более продажи зависят не только от типа товара, но и от магазина, поэтому в одном магазине один товар может продаваться лучше, в другом хуже. 
* количество товаров всех типов в каждом магазине одинаковое - то есть в магазинах одинаковый ассортимент (проверено итерацией `df.item[(df.store == j) & (df.item == i)].count()`, где i - итерация по товарам, j - итерация по магазинам), поэтому нельзя сейчас сделать вывод о том, что продажи какого то товара в одном магазине больше, потому что в этом магазине есть этот товар, а в других нет.
* продажи растут с каждым годом.

В связи с последним выводом, можно посмотреть, растут ли продажи с измением года во всех магазинах, или для всех ли позиций. Для этого можно разделить данные по магазину и типам товаров:

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(321)
for i in range(1, 11):
    plt.plot(train_df.sales[(train_df.store == i) ].groupby(train_df.date.dt.year).sum(), label='Store ' + str(i))
plt.title('Year, sum by store.')
plt.xticks(range(2013, 2018))
plt.grid()
plt.legend(loc='upper left')

plt.subplot(322) 
for i in range(1, 51):
    plt.plot(train_df.sales[(train_df.item == i) ].groupby(train_df.date.dt.year).sum(), label='Item' + str(i))
plt.title('Year, sum by item.')
plt.xticks(range(2013, 2018))
plt.grid()

plt.subplot(323)
for i in range(1, 11):
    plt.plot(train_df.sales[(train_df.store == i)].groupby(train_df.date.dt.month).sum(), label='Store ' + str(i))
plt.title('Month, sum by store.')
plt.xticks(range(1, 13))
plt.grid()
plt.legend(loc='upper left')

plt.subplot(324)
for i in range(1, 51):
    plt.plot(train_df.sales[(train_df.item == i)].groupby(train_df.date.dt.month).sum(), label='Item ' + str(i))
plt.title('Month, sum by item.')
plt.xticks(range(1, 13))
plt.grid()

plt.subplot(325)
for i in range(1, 11):
    plt.plot(train_df.sales[(train_df.store == i)].groupby(train_df.date.dt.dayofweek).sum(), label='Store ' + str(i))
plt.title('Day of week, sum by store.')
plt.grid()
plt.legend(loc='upper left')

plt.subplot(326)
for i in range(1, 51):
    _ = plt.plot(train_df.sales[(train_df.item == i)].groupby(train_df.date.dt.dayofweek).sum(), label='Item ' + str(i))
plt.title('Day of week, sum by item.')
plt.grid()

Зависимости практически одинаковые для всех магазинов и типов товаров (форма кривой), изменяется только абсюлютные значения, что не дает возможности выделить какой то особый магазин или товар, для которых тренд был бы особым.

Аналогично, для разреза по магазинам наблюдается одинаковый тренд, как в зависимости от месяца, так и в зависимости от дня недели.
Более того, расположение линий по магазинам и товарам сохраняется при разных временных разрезах (то есть от мин к макс для магазинов: магазин 7, магазин 6, магазин 1, итд - одинаково как в разрезах для месяца, дня недели или года; аналогично и для типа товара).

### 1.3 Scatter plot

По другому можно визуализировать данные используя три оси. Такая визуализация может быть полезной, так как позволит понять, можно ли выделить какие то кластеры в данных или нет. При наличии кластеров, возможно использование методов кластеризации, с последующим решением регрессионной задачи. Поскольку у нас имеется только 4 параметра: магазин, тип товара, продажи и время (месяц, год или день недели), наши данные можно визуализировать в трех плоскостях, используя разные цвета для точек в зависимости от магазина:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

mark = ['', 'o', '^', '.', 'x', '^', 'o', '^', '.', 'x', 'o']
for i in range(1, 4):
    ax.scatter(train_df.date.dt.month[train_df.store == i+2], 
               train_df.item[train_df.store == i+2], 
               train_df.sales[train_df.store == i+2], marker=mark[i])

plt.xticks(range(1, 13))
plt.yticks(range(1, 55, 5))

ax.set_xlabel('Month')
ax.set_ylabel('item')
ax.set_zlabel('Sales')  
ax.view_init(elev=10., azim=45)

Как видно, данные практически полностью перекрываются, поскольку находятся в одном диапазоне, и в них сложно выделить кластеры (и то используются только 3 магазина).

### 1.4 PCA

Попробуем выделить главные компоненты, чтобы понять, возможно ли с их помощью разделить данные.

Поскольку метод выделения главных компонент требует стандартизации данных, на финальном графике не будет видно полос, представляющих дискретные данные.

In [0]:
pca = PCA(n_components=3)
pca_df = train_df.drop(['date'], 1).copy()
pca_df = pd.DataFrame(preprocessing.StandardScaler().fit_transform(pca_df.values),
                      columns=pca_df.columns, index=pca_df.index)
pca_result = pca.fit_transform(pca_df)

pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
pca_df['pca-three'] = pca_result[:,2]

# For reproducability of the results
np.random.seed(42)
rndperm = np.random.permutation(pca_df.shape[0])

In [ ]:
# 2d scatter
plt.figure(figsize=(16,10))
_ = sns.scatterplot(x="pca-one", y="pca-two",
                    hue="store",
                    palette=sns.color_palette("hls", 10),
                    data=pca_df.loc[rndperm, :],
                    alpha=0.3)

В двух осях данные выглядят смешанными. Посмотрим в 3х осях.

In [ ]:
# 3d scatter
ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(xs=pca_df.loc[rndperm,:]["pca-one"], 
           ys=pca_df.loc[rndperm,:]["pca-two"], 
           zs=pca_df.loc[rndperm,:]["pca-three"], 
           c=pca_df.loc[rndperm,:]["store"], 
           cmap='tab10')

ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

При использовании трех компонент возможно разделить данные так, что явно видно определенные кластеры.
Преобразуем данные используя PCA с пятью компонентами для тренировки моделей:

In [0]:
pca_targ = train_df['sales']
pca_df = train_df.drop(['date'], 1).copy()
pca_df = pd.DataFrame(preprocessing.StandardScaler().fit_transform(pca_df.values),
                      columns=pca_df.columns, index=pca_df.index)

pca = PCA(n_components=5)
pca_result = pca.fit_transform(pca_df)
pca_features = pd.DataFrame(pca_result)

X_test = pd.DataFrame(preprocessing.StandardScaler().fit_transform(test_df.drop(['date'], 1).values),
                      index=test_df.drop(['date'], 1).index,
                      columns=test_df.drop(['date'], 1).columns)
pca_test = pd.DataFrame(pca.fit_transform(X_test))

# 2. Preparation

### 2.1 Training and validation data

На стадии подготовки данных разделим тренировочный датасет на фичи и лейблы. Более того, для фич необходимо произвести стандартизацию (мат. ожидание = 0, среднеквадратичное отклонение = 1), поскольку многие модели из библиотеки sklearn предполагают, что данные стандартизированы.

Переменная folds будет использоваться для свертках при кросс валидации. Выбираем 5 разделений, для сокращения времени тренировки моделей, но за счет увеличения числа разделений иногда можно улучшить производительность моделей.

In [0]:
features = train_df.drop(['date', 'sales'], 1)
targets = train_df['sales']

features = preprocessing.StandardScaler().fit_transform(features.values)

X_test = pd.DataFrame(preprocessing.StandardScaler().fit_transform(test_df.drop(['date'], 1).values),
                      index=test_df.drop(['date'], 1).index,
                      columns=test_df.drop(['date'], 1).columns)

folds = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

### 2.2 Class estimator (sklearn)

Создадим класс для использования с моделями sklearn.

In [0]:
class SKLEstimator:    

    def __init__(self, estimator, param_grid, folds=folds, verbose=0):
        self.grid = model_selection.GridSearchCV(estimator=estimator, param_grid=param_grid, scoring='neg_mean_squared_error', cv=folds, verbose=verbose, n_jobs=-1)
        self.estimator = estimator
    
  
    def fit_grid(self, X, y=targets, verbose=False):
        self.grid.fit(X, y)
        if verbose:
            print('')
            print('Best score: {:.2f}'.format(math.sqrt(abs(self.grid.best_score_))))
            print('Best parameters: {}'.format(self.grid.best_params_))
      
    def get_best_estimator(self):
      
        return self.grid.best_estimator_
 

    def best_params(self):
      
        return self.grid.best_params_
      
      
    def train_estimator(self, X, y=targets, folds=folds, verbose=False):
      
        scores = []

        for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):

            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]

            self.grid.best_estimator_.fit(X_train, y_train)
            y_pred_valid = self.grid.best_estimator_.predict(X_valid).reshape(-1)

            scores.append(metrics.mean_squared_error(y_valid, y_pred_valid))
      
        if verbose:
            print('Mean of CV MSE scores (on train/valid set): {0:.2f}'.format(math.sqrt(abs(np.mean(scores)))))  
      
        return scores
    
    def predict_targets(self, X_test):
      
        return self.grid.best_estimator_.predict(X_test)    

Функция для сдачи результатов

In [0]:
submit_df = subm_df_raw.copy()
def submit_preds(estimator, name, X_test=X_test, submit_df=submit_df, to_csv=False):

    submit_df['sales'] = estimator.predict(X_test).astype(int) 
    name_csv = name + ".csv"
    
    if to_csv:
        submit_df.to_csv(name_csv, index=False)
    
    return submit_df

# 3. Models

## 3.1 Sklearn models

В данном разделе попробуем применить несколько регрессионных моделей. Поскольку данные довольно однородные и пока не удалось выделить какие то особенности магазинов или типов товаров (хотя не исключаю, что их можно найти), пока сложно сделать вывод о применимости какой либо модели, поэтому методом проб и ошибок, постараемся найти лучшую модель.

### 3.1.1 Linear regression model

In [ ]:
grid_linreg = {'fit_intercept': [True, False]}
linreg_estim = SKLEstimator(linear_model.LinearRegression(), grid_linreg)
linreg_estim.fit_grid(features, verbose=True)

Модель дает средний результат (не слишком ужасный, но и далеко от хорошего), на тестовом сете при LB score не ниже 25.

Ниже для примера использования показаны модели линейной регресстии, использующие l1, l2 и l1+l2 регуляризацию. Итог на тестовом сете, не ниже 25. 

*В коде ниже тренировка моделей закомментирована, чтобы не тратить время на прогон скрипта*

In [0]:
### Lasso estimator
grid_lasso = {'alpha': [1e-3, 1e-2, 1e-1, 1, 2],
              'fit_intercept': [True, False],
              'tol': [1e-4, 1e-3, 1e-1, 5e-1]}
lasso_estim = SKLEstimator(linear_model.Lasso(), grid_lasso)
#lasso_estim.fit_grid(features, verbose=True)

In [0]:
### Ridge estimator
grid_ridge = {'alpha': [1e-3, 1e-2, 1e-1],
              'fit_intercept': [True, False],
              #'tol': [1e-4, 1e-3, 1e-1, 5e-1],
              'solver': ['svd', 'lsqr']}
ridge_estim = SKLEstimator(linear_model.Ridge(), grid_ridge, verbose=1)
#ridge_estim.fit_grid(features, verbose=True)

In [0]:
### Elnet estimator
grid_elnet = {'alpha': [1e-3, 1e-1, 1, 2],
              'l1_ratio': [1e-3, 1e-2, 5e-1],
              'fit_intercept': [True, False]}
elnet_estim = SKLEstimator(linear_model.ElasticNet(), grid_elnet, verbose=1)
#elnet_estim.fit_grid(features, verbose=True)

### 3.1.2 KNN regressor model

Модели на основе К ближайших соседей приводят к перетренированности модели. Результат на тренировочном сете хороший (MSE 11 и 8), но на тестовом сете показатель не меньше 40!

*В коде ниже тренировка моделей закомментирована, чтобы не тратить время на прогон скрипта*

*В комментариях - показатели моделей с разными параметрами*

In [0]:
grid_knn = {'n_neighbors': [3, 5, 10],
            'weights': ['uniform', 'distance'],
            'algorithm': ['ball_tree', 'kd_tree']}
knn_estim = SKLEstimator(neighbors.KNeighborsRegressor(), grid_knn, folds=folds, verbose=1)
#knn_estim.fit_grid(features, verbose=True)

# Best score: 11.60394831102819
# Best parameters: {'n_neighbors': 3, 'weights': 'uniform'}
  
# Best score: 8.826924270536434
# Best parameters: {'algorithm': 'kd_tree', 'n_neighbors': 10, 'weights': 'distance'}

# Best score: 8.826924270536434
# Best parameters: {'n_neighbors': 10, 'weights': 'distance'}

### 3.1.3 sklearn.tree.DecisionTreeRegressor

Поскольку деревья решений используются в многих алгоритмах бустинга, можно попробовать использовать деревья без бустинга (разрешая им расти полностью - параметр `'max_depth': [None]`).

Так же использовались фичи полученные выделением главных компонент (PCA), итоговый результат оказался хуже.

*В коде ниже тренировка моделей закомментирована, чтобы не тратить время на прогон скрипта*

*В комментариях - показатели моделей с разными параметрами*

In [0]:
grid_dt = {'splitter': ['best', 'random'],
           'max_depth': [10, 20, 50],
           'min_samples_split': [10, 30],
           'min_samples_leaf': [5, 15],
           'min_weight_fraction_leaf': [0, 0.2],          
          }
dt_estim = SKLEstimator(tree.DecisionTreeRegressor(), grid_dt, verbose=1)
#dt_estim.fit_grid(features, verbose=True) # may take time to run!
#submit_df = submit_preds(dt_estim.get_best_estimator(), "DT_3", to_csv=True)

### PCA
#dt_estim = SKLEstimator(tree.DecisionTreeRegressor(), grid_dt, verbose=1)
#dt_estim.fit_grid(pca_features, pca_targ, verbose=True)
#submit_df = submit_preds(dt_estim.get_best_estimator(), "DT_pca1", pca_test, to_csv=True)


# Best score: 8.97
# Best parameters: {'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0, 'splitter': 'best'}

Дерево решений занимает время на тренировку (поскольку при возможности оно растет полностью), и при этом явно приводит к перетренированности модели: 

MSE на тренировочном сете = 9

LB score на тестовом сете > 25.

Однако, при лучшей настройки модели можно добиться лучших результатов.

### 3.1.4 sklearn.ensemble.RandomForestRegressor

Лес деревьев так же приводит к перетренированности модели. LB score не ниже 24.

Так же использовались фичи полученные выделением главных компонент (PCA), итоговый результат оказался хуже.

*В коде ниже тренировка моделей закомментирована, чтобы не тратить время на прогон скрипта*

*В комментариях - показатели моделей с разными параметрами*

In [0]:
grid_rf = {'n_estimators': [10, 100],
           'max_depth': [2, 3, None],
           'min_samples_split': [2, 30],
           'min_samples_leaf': [1, 5],
           'min_weight_fraction_leaf': [0, 0.5]}

grid_rf = {'n_estimators': [100],
           'max_depth': [None],
           'min_samples_split': [2],
           'min_samples_leaf': [5],
           'min_weight_fraction_leaf': [0]}

rf_estim = SKLEstimator(ensemble.RandomForestRegressor(), grid_rf, folds=folds, verbose=1)
#rf_estim.fit_grid(features, verbose=True) # may take time to run!


### PCA:
#rf_estim = SKLEstimator(ensemble.RandomForestRegressor(), grid_rf, folds=folds, verbose=1)
#rf_estim.fit_grid(pca_features, pca_targ, verbose=True)
#submit_df = submit_preds(rf_estim.get_best_estimator(), "RF_pca1", pca_test, to_csv=True)

### Run 1:
#Best score: 7.652671941592358
#Best parameters: {'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0, 'n_estimators': 100}

## 3.2 Boosting, ensemble models

### 3.2.1 XGBoost

Сейчас популярен метод бустинга из библиотеки xgboost, из за его высокой производительности и сравнительно низкого времени тренировки. Попробуем несколько моделей с разными параметрами.

In [0]:
features_xgb = pd.DataFrame(preprocessing.StandardScaler().fit_transform(train_df.drop(['date', 'sales'], 1).values),
                            index=train_df.drop(['date', 'sales'], 1).index, 
                            columns=train_df.drop(['date', 'sales'], 1).columns)

dtrain = xgb.DMatrix(features_xgb, label=targets, feature_names=list(features_xgb.columns))
dtest = xgb.DMatrix(X_test, feature_names=list(X_test.columns))

Всего было протестировано 10 моделей XGBoost с разными параметрами, на данный момент лучший результат LB private / public: 24.33419 / 14.78021. Заметно ухудшение результатов при переходе от public score к private score, что может говорить о перетренированности модели. Ниже приведено несколько моделей. Можно проводить дальнейшную настройку для улучшения результатов.

In [0]:
# LB private / public: 24.77016 / 15.06425
xgb_grid = {'max_depth': 3, 'eta': 5e-1} # 'subsample': 1
#xgb_estim = xgb.train(xgb_grid, dtrain, 1500)
#submit_df = submit_preds(xgb_estim, "XGB_2", X_test=dtest, to_csv=True)

In [0]:
# LB private / public: 24.21677 / 14.81929
xgb_grid = {'max_depth': 5, 'eta': 5e-1}
#xgb_estim = xgb.train(xgb_grid, dtrain, 1500)
#submit_df = submit_preds(xgb_estim, "XGB_4", X_test=dtest, to_csv=True)

In [0]:
# LB private / public: 24.20192 / 14.78730
xgb_grid = {'max_depth': 5, 'eta': 2e-1}
xgb_estim = xgb.train(xgb_grid, dtrain, 1500)
submit_df = submit_preds(xgb_estim, "XGB_1", X_test=dtest, to_csv=True)

In [0]:
# LB private / public: 24.33419 / 14.78021
xgb_grid = {'max_depth': 5, 'eta': 1e-1}
#xgb_estim = xgb.train(xgb_grid, dtrain, 1500)
#submit_df = submit_preds(xgb_estim, "XGB_9", X_test=dtest, to_csv=True)

In [ ]:
# LB private / public: 
xgb_grid = {'booster': 'dart', 
            'max_depth': 5, 
            'eta': 5e-1}
#xgb_estim = xgb.train(xgb_grid, dtrain, 1500)
#submit_df = submit_preds(xgb_estim, "XGB_11", X_test=dtest, to_csv=True)

При использовании вместо фич гланых компонент результат получается хуже (не ниже 56! ((  ). Возможно произведен некорректный выбор гланых компонент, либо необходимо по другому настраивать модель бустинга.

In [0]:
dtrain_pca = xgb.DMatrix(pca_features, label=pca_targ)
dtest_pca = xgb.DMatrix(pca_test)

In [0]:
# LB private / public: 56.64127 / 56.36665
xgb_grid = {'max_depth': 5, 'eta': 2e-1}
#xgb_estim = xgb.train(xgb_grid, dtrain_pca, 1500)
#submit_df = submit_preds(xgb_estim, "XGB_pca_1", X_test=dtest_pca, to_csv=True)

### 3.2.2 sklearn.ensemble.ExtraTreesRegressor

Данная модель показала пока лучшие результаты для LB private score = 23.41. Несмотря на то, что модель все еще сильно перетренирована, так как private score в два раза отличается от тренировочной метрики (хотя может там используется другая метрика, а не MSE), на тренировку не уходит много времени, так как деревья ограничены 3, 4 узлами. Данную модель можно улучшать далее путем настройки большего числа доступных параметров, наравне с XGBoost для улучшения результатов.

Датасеты PCA снова не показали хороших результатов.

In [0]:
# LB private / public: 23.41066 / 14.79618
grid_extree = {'n_estimators': [100],
               'max_depth': [None],
               'min_samples_split': [2],
               'min_samples_leaf': [1],
               'bootstrap': [False],
              }
extree_estim = SKLEstimator(ensemble.ExtraTreesRegressor(n_jobs=-1), grid_extree, verbose=1)
extree_estim.fit_grid(features, verbose=True) # may take time to run!
submit_df = submit_preds(extree_estim.get_best_estimator(), "extree_1", to_csv=True)


### PCA: 
#extree_estim = SKLEstimator(ensemble.ExtraTreesRegressor(n_jobs=-1), grid_extree, verbose=1)
#extree_estim.fit_grid(pca_features, pca_targ, verbose=True)
#submit_df = submit_preds(extree_estim.get_best_estimator(), "extree_pca_2", pca_test, to_csv=True)

#Best score: 12.03
#Best parameters: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

### 3.2.3 sklearn.ensemble.GradientBoostingRegressor

Модель градиентного бустинга sklearn несколько более привычная чем xgboost и позволяет довольно тонко настраивать модель за счет использования множества инструментов библиотеки sklearn (например, изменение метода выбора сверток с KFold на StratifiedKFold), и в данной ситуации модель показывает хорошие результаты (хоть и далекие от идеала), которые можно улучшать:

In [0]:
# LB private / public: 24.61912 / 15.31381
grid_gboost = {'learning_rate': [1e-2, 1e-1, 5e-1, 1],
               'n_estimators': [100, 200],
               'min_samples_split': [2, 3],
               'min_samples_leaf': [1, 3],
               'max_depth': [3, 5],
              }

best_params = {'learning_rate': [5e-1],
               'max_depth': [5],
               'min_samples_leaf': [3],
               'min_samples_split': [2],
               'n_estimators': [200]}

#gboost_estim = SKLEstimator(ensemble.GradientBoostingRegressor(), best_params, verbose=1)
#gboost_estim.fit_grid(features, verbose=True) # may take time to run!

#Best score: 16.00
#Best parameters: {'learning_rate': 0.1, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}

# 4. Conclusions

* Определены следующие модели, которые можно настаивать далее для поиска лучших параметов: XGBoost, GradientBoostingRegressor, ExtraTreesRegressor.

* Выделение главных компонент хоть и выглядело обещающе, за счет того, что позволяло разделить данные на кластеры (не стояла задача снижения размерности данных, поскольку мы не имеем много фич, а скорее задача трансформации фич с использованием главных компонент). Возможна настройка алгоритма поиска главных компонент, либо можно использовать некоторые компоненты как отдельные фичи вместе с другими основными. Для этого, после тренировки моделей, можно оценить важность фич за счет аргумента feature_importance, доступного в некоторых моделях (например .feature_importances_ для XGBoost, GradientBoostingRegressor).